In [ ]:
#3
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import imageio

In [ ]:
africa_data = pd.read_csv('atlas_africa.csv', low_memory=False)

resistant_data = africa_data[africa_data['Status'] == 'Resistant']

species_list = resistant_data['Species'].unique()

In [ ]:
# Initialize an empty list to store file paths for GIF generation
image_paths = []

# Define the number of rows and columns for the subplots
rows, cols = 3, 3
max_subplots = rows * cols 

for species in species_list:
    species_data = resistant_data[resistant_data['Species'] == species]

    # Calculate resistance proportions by age group and antibiotic class
    age_resistance = species_data.groupby(['Age Group', 'antibiotics_class']).size() / species_data.groupby('Age Group').size()

    age_resistance_df = age_resistance.reset_index(name='Resistance Rate')

    # Check if there are any antibiotic classes for this species
    unique_classes = age_resistance_df['antibiotics_class'].unique()
    if len(unique_classes) == 0:
        continue

    # Split the antibiotic classes into batches of 9 (for 3x3 grid)
    for i in range(0, len(unique_classes), max_subplots):
        batch_classes = unique_classes[i:i+max_subplots]

        # Create a figure with 3x3 subplots and adjust vertical spacing
        fig = make_subplots(rows=rows, cols=cols, 
                            subplot_titles=batch_classes,
                            vertical_spacing=0.3)  # Increase vertical spacing

        # Plot each age group per antibiotic class
        for idx, antibiotic_class in enumerate(batch_classes):
            class_data = age_resistance_df[age_resistance_df['antibiotics_class'] == antibiotic_class]
            
            # Sort by descending resistance rate
            class_data_sorted = class_data.sort_values('Resistance Rate', ascending=False)

            # Determine the current subplot position (row, col)
            row = idx // cols + 1
            col = idx % cols + 1

            # Create bar trace
            trace = go.Bar(x=class_data_sorted['Age Group'], 
                           y=class_data_sorted['Resistance Rate'], 
                           name=antibiotic_class,
                           marker=dict(color=px.colors.qualitative.Plotly[idx % len(px.colors.qualitative.Plotly)]))

        
            fig.add_trace(trace, row=row, col=col)

            fig.update_xaxes(tickangle=45, row=row, col=col)

        fig.update_layout(title_text=f'Resistance by Age Group for {species}', barmode='stack',
                          showlegend=False, height=900, width=1200)

        # Save each figure as a PNG for GIF creation
        image_path = f'resistance_{species}_{i // max_subplots + 1}.png'
        fig.write_image(image_path)
        image_paths.append(image_path)

# Create GIF from the saved images
with imageio.get_writer('resistance_by_agegroup.gif', mode='I', duration=3000) as writer:
    for image_path in image_paths:
        image = imageio.imread(image_path)
        writer.append_data(image)

plt.imshow(imageio.imread('resistance_by_agegroup.gif'))
plt.axis('off')
plt.show()
